# Single Cell sequencing (scRNA-seq)

This notebook consist of:
- Data QC (at cell and genes level)
- Visualisation (embedding)
- Machine learning 

We utilized the PBMC (peripheral blood mononuclear cells)datasets and mostly utilized Scanpy (equivalent to Seurat in R)

# Environment setup 
The scRNA usually required multiple files which make it better to run it inside a conda/mamba environment to prevent any damage to the local computer.

In [ ]:
# Environment setup; succesful loading here should mean all dependencies correctly installed #
import numpy as np
import pandas as pd
import scanpy as sc

from scipy import stats
from scipy.cluster.hierarchy import dendrogram

import seaborn as sbn
import matplotlib.pyplot as plt

from gprofiler import GProfiler

from natsort import natsorted
from sklearn.cluster import AgglomerativeClustering

In [ ]:
# Suppress warnings for tidy representation of the notebook   
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'
    
def print_msg(msg_lst: list, sep='\t', color='\033[91m'):
    msg = ""
    for i in msg_lst:
        msg += str(i) + sep

    print(color + "-" * 80 + bcolors.ENDC)
    print(color + msg.center(80, ' ') + bcolors.ENDC)
    print(color + '-' * 80 + bcolors.ENDC)

## Loading the data

In [ ]:
#############################
#   Loading the Data   # 
#############################
pbmc = sc.read_10x_mtx(
    'data/filtered_gene_bc_matrices/hg19/',  # the directory with the `.mtx` file
    var_names='gene_symbols',                # use gene symbols for the variable names (variables-axis index)
    cache=True)                              # write a cache file for faster subsequent reading

In [ ]:
# Understand the data structure
print(pbmc.X.getnnz(axis = 1)) #2700 x 32738
print(pbmc.var)               #32738 genes
print(pbmc.obs)               #2700 cells
print(pbmc.uns)               #unstructured annotations

# QC
## QC at cell level 
There is usually some variations between cells due to poor separation of oil droplet (the scRNA-seq usually used droplet-based method) and sequencing error. 

### QC metrics using ribosomal cell 
High expression of ribosomal genes indicates failure to capture other expressed genes within the cell

In [ ]:
# Create an observation metric info to add the total reads per cell
obs_metrics = pd.DataFrame(index=pbmc.obs_names) ## Get the identifiers

print("Total Rb genes ", sum(pbmc.var_names.str.startswith(('RPL', 'RPS')))) # Number of ribosomal genes

# Calculate QC metrics for ribosomal genes 
pbmc.var['rb'] = pbmc.var_names.str.startswith(('RPL', 'RPS'))
sc.pp.calculate_qc_metrics(
    pbmc,
    qc_vars=['rb'],
    percent_top=None,
    log1p=False,
    inplace=True
)

# Calculate the RB cells >50%
print("total RB cells >50%:", (pbmc.obs['pct_counts_rb'] > 50).sum())

### QC metrics using mitochondrial cells
High mitochondrial expression means the cells is dying or almost reach senescences stage, which need to be excluded.

In [ ]:
print("Total Mt genes ", sum(pbmc.var_names.str.startswith('MT-'))) # Number of mitochondrial genes

pbmc.var["MT"] = pbmc.var_names.str.startswith('MT')

sc.pp.calculate_qc_metrics(
    pbmc,
    qc_vars=['MT'],
    percent_top=None,
    log1p=False,
    inplace=True
)

print("total MT cells >5%:", (pbmc.obs['pct_counts_MT'] > 5).sum())


## QC at genes level
Some low amplification might be missed during sequencing which result to lot of zeros in the cell.

See how many genes are expressed in no cells

In [ ]:
# get number of non-zero values in each row (gene) and add to annData object. 
pbmc.var['cells_per_gene'] = pbmc.X.getnnz(axis = 0) 
genes_in_no_cells = sum(pbmc.var['cells_per_gene'] == 0)
print('There are', genes_in_no_cells, 'genes expressed in no cells.')

In [ ]:
pbmc_copy = pbmc.copy()

# Normalize each cell to a total of 1e4 reads
sc.pp.normalize_total(pbmc_copy, target_sum=1e4)

# Scale it the data to unit variance and zero mean
sc.pp.scale(pbmc_copy)

# Log transform the data
sc.pp.log1p(pbmc_copy) #NOTE this adds a +1 pseudocount to prevent -infinity for log(0)

# Run PCA to reduce the dimensionality of the data
sc.tl.pca(pbmc_copy, svd_solver='arpack')


# Highly variable genes
This is important where they can be our marker genes to distinguish our cells (i.e T cell is CD4)

In [ ]:
sc.pp.highly_variable_genes(pbmc_copy, min_mean=0.0125, max_mean=3, min_disp=0.5)

print("Number of highly variable genes selected: ", sum(pbmc_copy.var['highly_variable'].values))

sc.pl.highly_variable_genes(pbmc_copy)

# Visualisation 

In [ ]:
# Using PCA to see the variance explained by each principal component

sc.pl.pca(pbmc_copy, color='LYZ')
sc.pl.pca(pbmc_copy, color='CD4', components='2,3')

In [ ]:
# Using UMAP for visualisation
sc.pp.neighbors(pbmc_copy, n_neighbors=10, n_pcs=40)
sc.tl.umap(pbmc_copy)
sc.pl.umap(pbmc, color=['LYZ','CD3E','MS4A1'] )

In [ ]:
# Use t-SNE for visualisation
sc.tl.tsne(pbmc_copy, random_state = 2)
sc.pl.tsne(pbmc_copy, color=['LYZ','CD3E','MS4A1'] )

## Clustering

### k-means clustering

In [ ]:
from sklearn.cluster import KMeans

data_pca = pbmc_copy.obsm['X_pca']

# kmeans clustering
kmeans_clust = KMeans(n_clusters = 3).fit(data_pca) 

plt.scatter(data_pca[:, 0], data_pca[:, 1], c=kmeans_clust.labels_)

### Hierarchial clustering

In [ ]:
HC_clust = AgglomerativeClustering(n_clusters=3, metric='euclidean', linkage='ward').fit(data_pca)

plt.scatter(data_pca[:, 0], data_pca[:, 1], c=HC_clust.labels_)

# Machine learning

In here, we only try 2 machine learning which are random forest and MLP.

We didn't do the hyperparameter tuning for both ML which can be a limitation.

In [ ]:
# Library 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, accuracy_score, f1_score, precision_score, recall_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier


## Additional QC

In [ ]:
# We did additional QC which is to choose top 2000 highly variable genes for analysis
sc.pp.highly_variable_genes(pbmc_copy, n_top_genes=2000)

train_pbmc = pbmc_copy[:, pbmc_copy.var['highly_variable']]

# visualize the highly variable genes
sc.tl.rank_genes_groups(train_pbmc, groupby="cell-types", method="wilcoxon")
sc.set_figure_params(scanpy=True, dpi=100, fontsize=14)
sc.pl.rank_genes_groups(train_pbmc, n_genes=10, sharey=False)

## Encode the data

In [ ]:
# Pre-processing encoder
X = train_pbmc.X

# Encode the Y (for easy use in ML models)
mapping = {"CD56+ NK" : 1, 
           "CD19+ B": 2, 
           "CD4+/CD45RA+/CD25- Naive T" : 3, 
           "Dendritic":4}

train_pbmc.obs["cell-encode"] = train_pbmc.obs["cell-types"].map(mapping)

print(train_pbmc.obs[["cell-types", "cell-encode"]].head())

#Y IS 
Y = train_pbmc.obs["cell-encode"]

In [ ]:
# Train test split
X_train, X_test, y_train, y_test = train_test_split(
    X, Y, test_size=0.30, random_state=42, stratify=Y
)

## Random forest

In [ ]:
#Classifier for random forest 
rf = RandomForestClassifier(random_state = 42, class_weight="balanced")

rf.fit(X_train, y_train)
y_train_pred = rf.predict(X_test)

inv_map = {v: k for k, v in mapping.items()}
target_names = [inv_map[i] for i in sorted(inv_map.keys())]

print("=== RF Results ===")
print("Accuracy:", accuracy_score(y_test, y_train_pred))
print("Macro F1:", f1_score(y_test, y_train_pred, average="macro"))
print("Precision:", precision_score(y_test, y_train_pred, average="macro"))
print("Recall:", recall_score(y_test, y_train_pred, average="macro"))

print(classification_report(y_test, y_train_pred, target_names=target_names))

## MLP

In [ ]:
# MLP classifier 
mlp = MLPClassifier(random_state=42)

mlp.fit(X_train, y_train)
y_mlp_pred = mlp.predict(X_test)

print("=== MLP Results ===")
print("Accuracy:", accuracy_score(y_test, y_mlp_pred))
print("Macro F1:", f1_score(y_test, y_mlp_pred, average="macro"))

print(classification_report(y_test, y_mlp_pred, target_names=target_names))

## Test data

In [ ]:
# Import the data
test_pbmc = sc.read_h5ad("Test_dataset.h5ad")

print(test_pbmc.obs["cell-types"].value_counts())


### QC

In [ ]:
# Remove non-important cell types
# Mitochondrial cells
test_pbmc.var["mt"] = test_pbmc.var_names.str.startswith('MT')
sc.pp.calculate_qc_metrics(test_pbmc, qc_vars=["mt"], inplace=True)
test_pbmc = test_pbmc[test_pbmc.obs["pct_counts_mt"] < 5].copy()

# Ribosomal cells
test_pbmc.var["rb"] = (
    test_pbmc.var_names.str.startswith("RPL") |
    test_pbmc.var_names.str.startswith("RPS")
)
sc.pp.calculate_qc_metrics(test_pbmc, qc_vars=["rb"], inplace=True)
test_pbmc = test_pbmc[test_pbmc.obs.pct_counts_mt < 10, :].copy()


# Normalize
sc.pp.normalize_total(test_pbmc, target_sum=1e4)
# Log transform 
sc.pp.log1p(test_pbmc)
# Choose top 2000 highly variables genes 
sc.pp.highly_variable_genes(test_pbmc, n_top_genes=2000, subset=True)
# Scale 
sc.pp.scale(test_pbmc)
# PCA
sc.tl.pca(test_pbmc, svd_solver='arpack')
# Get the X and Y


In [ ]:
# Encoding
# X and Y define 
X_mys_test = test_pbmc.X

# Encode the Y 
mapping_test = {"NK cell" : 1, 
           "B cell": 2, 
           "CD4+ T cell" : 3, 
           "CD8+ T cell": 3,
           "Other T": 3,
           "cDC":4,
           "pDC":4,
           "CD16+ Monocyte":5,
           "CD14+ Monocyte" :5,
           "Platelet":6,
           "Plasmablast":6}

inv_test = {1: "NK cell",
            2: "B cell",
            3: "T cell",
            4: "Dendritic",
            5: "Monocytes",
            6: "Others"}

test_pbmc.obs["cell-encode"] = test_pbmc.obs["cell-types"].map(mapping_test)

#Y IS 
Y_mys_encode = test_pbmc.obs["cell-encode"]

In [ ]:
# Exclusion of some cell types not in training data
to_exclude = ["Monocytes", "Others"]
filtered_pbmc = test_pbmc[~test_pbmc.obs["encode-reverse"].isin(to_exclude)].copy()

In [ ]:
# Redefine X and Y
# X and Y define 
X_filt_test = filtered_pbmc.X

#Y IS 
Y_filt_encode = filtered_pbmc.obs["cell-encode"]

print(filtered_pbmc.obs)

## Random forest

In [ ]:
y_filt_pred = rf.predict(X_filt_test)

print("=== RF Results ===")
print("Accuracy:", accuracy_score(y_filt_pred, Y_filt_encode))
print("Macro F1:", f1_score(y_filt_pred, Y_filt_encode, average="macro"))

print(classification_report(y_filt_pred, Y_filt_encode, target_names=target_names))

## MLP

In [ ]:
y_filt_mlp_pred = mlp.predict(X_filt_test)

filtered_pbmc.obs["mlp-pred"] = y_filt_mlp_pred

print("=== MLP Results ===")
print("Accuracy:", accuracy_score(y_filt_mlp_pred, Y_filt_encode))
print("Macro F1:", f1_score(y_filt_mlp_pred, Y_filt_encode, average="macro"))
print("Precision:", precision_score(y_filt_mlp_pred, Y_filt_encode, average="macro"))
print("Recall:", recall_score(y_filt_mlp_pred, Y_filt_encode, average="macro"))

print(classification_report(y_filt_mlp_pred, Y_filt_encode, target_names=target_names))